In [ ]:
#!pip install anvil-uplink

In [ ]:
import numpy as np
import pandas as pd
import csv
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,classification_report

#for pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

import anvil.server

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_excel("../input/raw-data/raw_data.xlsx")

In [ ]:
data

In [ ]:
data = data.fillna(method='ffill')

In [ ]:
data['Disease'] = data['Disease'].str.split("^",expand = True)[0]
data['Symptom'] = data['Symptom'].str.split("^",expand = True)[0]

In [ ]:
data['Disease'] = data['Disease'].str.split("_",expand = True)[1]
data['Symptom'] = data['Symptom'].str.split("_",expand = True)[1]

In [ ]:
data_new = data.copy()

In [ ]:
def renew_data(data, col1, col2):
    i = 1
    while i<data.shape[0]:
        if data[col1].iloc[i]==data[col1].iloc[i-1]:
            data[col2].iloc[i] = data[col2].iloc[i-1]+" "+data[col2].iloc[i]
        i = i +1
    return data

In [ ]:
renew_data(data_new,'Disease','Symptom')

In [ ]:
data_new.groupby('Disease').describe()

In [ ]:
cv = CountVectorizer()

In [ ]:
X = data_new['Symptom']
Y = data_new['Disease']

In [ ]:
X = cv.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.3,random_state=101)

In [ ]:
nb = MultinomialNB()

In [ ]:
nb.fit(X_train,y_train)

In [ ]:
predictions = nb.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

In [ ]:
z = []
t = input().lower()
z.append(t)
z = pd.DataFrame(z, columns=['ABC'])
z = z['ABC']
z = cv.transform(z)
z_pred = nb.predict(z)
z_pred[0]

In [ ]:
anvil.server.connect('DLBMGK5KNJUILT25GCZD7AQK-E3R23QQLKE2LCHQY')
@anvil.server.callable
def classify_disease(sym):
    z = []
    z.append(sym)
    z = pd.DataFrame(z, columns=['ABC'])
    z = z['ABC']
    z = cv.transform(z)
    z_pred = nb.predict(z)
    return z_pred[0]

# **PIPELINE**

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [ ]:
X_pipe = data_new['Symptom']
Y_pipe = data_new['Disease']

In [ ]:
pipeline.fit(X_pipe,Y_pipe)

In [ ]:
pred = pipeline.predict(data_new['Symptom'])

In [ ]:
print(confusion_matrix(data_new['Disease'],pred))
print(classification_report(data_new['Disease'],pred))